<a href="https://colab.research.google.com/github/dgraselli/samples/blob/master/Practica_3_e7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys 
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)
root_path = 'drive/My Drive/FACULTAD/POSGRADO/Conceptos y Aplicaciones de Big Data/P3/' 
sys.path.append('/content/../LIB' + root_path)




Mounted at /content/drive


In [2]:
# Instalamos Spark para Python
"""
!pip install pyspark
import os

# Instalamos Java SDK 8
!apt-get install -y openjdk-8-jdk -qq > /dev/null      
!echo $(/usr/libexec/java_home -v 1.8)

#set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     
!echo 2 | update-alternatives --config java
"""


'\n!pip install pyspark\nimport os\n\n# Instalamos Java SDK 8\n!apt-get install -y openjdk-8-jdk -qq > /dev/null      \n!echo $(/usr/libexec/java_home -v 1.8)\n\n#set environment variable\nos.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     \n!echo 2 | update-alternatives --config java\n'

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "myProg")


In [4]:
clientes = sc.textFile(root_path +'/Banco/input/clientes/Clientes.txt')
prestamos = sc.textFile(root_path +'/Banco/input/prestamos/Prestamos.txt')
movimientos = sc.textFile(root_path +'/Banco/input/movimientos/Movimientos.txt')
cajas = sc.textFile(root_path +'/Banco/input/cajas_de_ahorro/CajasDeAhorro.txt')
#Cliente: <ID_Cliente, nombre, apellido, DNI, fecha de nacimiento, nacionalidad>
#CajaDeAhorro: <ID_Caja, ID_Cliente, saldo>
#Prestamos: <ID_Caja, cuotas, monto>
#Movimientos: <ID_Caja, monto, fecha>


In [5]:
from dateutil.relativedelta import relativedelta
import datetime

def continente(x):
  return 'EUR' if x in ['ESP','ITA'] else 'AME'

def edad(fn):
  y,m,d = fn.split('-')
  start_date = datetime.date(int(y), int(m), int(d))
  end_date = datetime.date.today()
  difference_in_years = relativedelta(end_date, start_date).years
  return difference_in_years

#edad('1977-07-21')

In [6]:

t_clientes = clientes.map(lambda x: x.split()) \
                     .map(lambda x: {"id_cte":x[0], 
                                     "name": x[1] + " " + x[2],
                                     "dni": x[3],
                                     "fn": x[4],
                                     "nac": x[5],
                                     "cont": continente(x[5])})

t_cajas  = cajas.map(lambda x: x.split()) \
                .map(lambda x: {"id_caja": x[0],
                                "id_cte": x[1],
                                "saldo": float(x[2])})   

t_prestamos  = prestamos.map(lambda x: x.split()) \
                        .map(lambda x: {"id_caja": x[0],
                                "cuotas": int(x[1]),
                                "monto": float(x[2])})   


In [7]:
# 7a) El banco tiene mas clientes europeos que americanos
t_clientes_ame = t_clientes.filter(lambda x: x['cont'] =='AME').map(lambda x: (x['id_cte'],x))
t_clientes_eur = t_clientes.filter(lambda x: x['cont'] =='EUR').map(lambda x: (x['id_cte'],x))

c_ame = t_clientes_ame.count()
c_eur = t_clientes_eur.count()

print("Americanos: {}, Europeos: {}".format(c_ame, c_eur)) 
print("7a) El banco tiene mas clientes europeos que americanos: {}".format(c_ame < c_eur))

Americanos: 22661, Europeos: 4425
7a) El banco tiene mas clientes europeos que americanos: False


In [8]:
# 7b) El promedio de edad de los clientes americanos es menor que el de los europeos

ame_sum, ame_count = t_clientes_ame.map(lambda x: (edad(x[1]['fn']),1)).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
eur_sum, eur_count = t_clientes_eur.map(lambda x: (edad(x[1]['fn']),1)).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
respuesta = ame_sum / ame_count <  eur_sum / eur_count
print("Promedio de edad de americanos: {:.2f}".format(ame_sum / ame_count))
print("Promedio de edad de europeos: {:.2f}".format(eur_sum / eur_count))
print("7b) El promedio de edad de los clientes americanos es menor que el de los europeos: {}".format(respuesta))

Promedio de edad de americanos: 48.71
Promedio de edad de europeos: 48.76
7b) El promedio de edad de los clientes americanos es menor que el de los europeos: True


In [9]:
# 7c) Los americanos deben más plata que los europeos (un cliente debe plata si la suma de montos de todas sus cajas de ahorro es negativa)
p_cte_caja = t_cajas.map(lambda x: (x['id_cte'],x))
ame = t_clientes_ame.join(p_cte_caja).map(lambda x: x[1][1]['saldo']).sum()
eur = t_clientes_eur.join(p_cte_caja).map(lambda x: x[1][1]['saldo']).sum()
print("Saldo AME: {:.2f}".format(ame))
print("Saldo EUR: {:.2f}".format(eur))
print("7c) Los americanos deben más plata que los europeos: {}".format(ame < eur))

Saldo AME: -18319419.83
Saldo EUR: -13785723.21
7c) Los americanos deben más plata que los europeos: True


In [10]:
# 7d) Los clientes americanos suelen sacar, en promedio, préstamos con mayor cantidad de cuotas que los europeos
p_caja_prestamos = t_prestamos.map(lambda x: (x['id_caja'],x))
ame_sum, ame_cnt = t_clientes_ame.join(p_cte_caja) \
      .map(lambda x: (x[1][1]['id_caja'], None)) \
      .join(p_caja_prestamos) \
      .map(lambda x: (x[1][1]['cuotas'],1)) \
      .reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))

eur_sum, eur_cnt = t_clientes_eur.join(p_cte_caja) \
      .map(lambda x: (x[1][1]['id_caja'], None)) \
      .join(p_caja_prestamos) \
      .map(lambda x: (x[1][1]['cuotas'],1)) \
      .reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))

ame_avg = ame_sum / ame_cnt
eur_avg = eur_sum / eur_cnt

print("Cuotas Prom AME: {:.2f}".format(ame_avg))
print("Cuotas Prom EUR: {:.2f}".format(eur_avg))
print("7d) Los clientes americanos suelen sacar, en promedio, préstamos con mayor cantidad de cuotas que los europeos: {}".format(ame_avg < eur_avg))

Cuotas Prom AME: 61.42
Cuotas Prom EUR: 60.18
7d) Los clientes americanos suelen sacar, en promedio, préstamos con mayor cantidad de cuotas que los europeos: False
